# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# guia selenium: https://selenium-python.readthedocs.io

navegador = webdriver.Chrome()
#("chromedriver.exe") para usar o chromedriver do local do arquivo

#Passo 1: pegando a cotação do dolar

navegador.get("https://www.google.com.br")
#By.XPATH=caminho do elemento pelo prompt de comando
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
#navegador.get("https://www.google.com/search?client=opera&q=valor+do+dolar&sourceid=opera&ie=UTF-8&oe=UTF-8")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
dolar=navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[3]/table/tbody/tr[3]/td[1]/input').get_attribute('value')
dolar=float(dolar)
print(dolar)

#Pegando Euro

navegador.get("https://www.google.com.br")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação Euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
euro=navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[3]/table/tbody/tr[3]/td[1]/input').get_attribute('value')
euro=float(euro)
print(euro)

#Pegando Ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")
ouro=navegador.find_element(By.XPATH,
    '//*[@id="comercial"]').get_attribute('value')
ouro=ouro.replace(",",".")
ouro=float(ouro)
print(ouro)

navegador.quit()


5.51
6.28
322.14


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
#Escanear planilha
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
#Criar nova planilha com valores atualizados
         #linha, coluna
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = dolar
#https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.loc.html
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = euro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = ouro


#tabela['Moeda'] in ["nome1", "nomeN"]


#Preço de Compra = Preço de compra * Cotação

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"] #"=D2*B2"

#Preço de venda = preço de venda * Margem

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"] #"=F2*E2"
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.55,5549.9445,1.40,7769.92230
1,Carro Renault,4500.00,Euro,6.34,28530.0000,2.00,57060.00000
2,Notebook Dell,899.99,Dólar,5.55,4994.9445,1.70,8491.40565
3,IPhone,799.00,Dólar,5.55,4434.4500,1.70,7538.56500
4,Carro Fiat,3000.00,Euro,6.34,19020.0000,1.90,36138.00000
5,Celular Xiaomi,480.48,Dólar,5.55,2666.6640,2.00,5333.32800
6,Joia 20g,20.00,Ouro,325.32,6506.4000,1.15,7482.36000


### Agora vamos exportar a nova base de preços atualizada

In [4]:
#exportando base de dados
tabela.to_excel("Produtos Novo.xlsx", index=False)